In [18]:
import os
from deepface import DeepFace
import cv2
import pandas as pd

# Caminho para as pastas de imagens
dataset_path = "dados/"

# Função para predizer o artista dado uma nova imagem
def predict_artist(image_path):
    # Analisar a imagem e comparar com o dataset
    print("Fazendo a predição...")
    result = DeepFace.find(
        img_path=image_path,
        db_path=dataset_path,
        model_name='VGG-Face',  # Passando o nome do modelo como string
        enforce_detection=False  # Ignora erros de detecção facial para garantir que a análise seja realizada
    )
    return result

if __name__ == "__main__":
    # Caminho para uma nova imagem para teste
    test_image = "dados/testes/brad.jpg"
    prediction = predict_artist(test_image)
    # Exibe o resultado

    if len(prediction[0]) != 1:
        artist = prediction[0].iloc[1, 0].split('/')[1].split('\\')[0]
        print("Artista previsto:", artist)
    else:
        print('Artista não encontrado na nossa base de dados.')


Fazendo a predição...
24-10-28 20:58:26 - Searching dados/brad.jpg in 1962 length datastore
24-10-28 20:58:27 - find function duration 1.960627555847168 seconds
Artista previsto: Brad Pitt


In [14]:
artist

82

In [20]:
prediction[0].head(3)

,identity,hash,target_x,target_y,target_w,target_h,source_x,source_y,source_w,source_h,threshold,distance
0,dados/brad.jpg,18bd06c793e59419a3d2e820edbf651f643d9417,424,46,193,193,424,46,193,193,0.68,0.000000
1,dados/Brad Pitt\033_46c95715.jpg,23dc2699c3dee0256d7d1e6a29eb065c724e134b,136,30,271,271,424,46,193,193,0.68,0.524978
2,dados/Brad Pitt\018_136dbb40.jpg,0419e714ebc82c5439f564621de5d46886ef6cd9,24,47,109,109,424,46,193,193,0.68,0.525117


In [32]:
prediction[0].iloc[1, 0].split('/')[1].split('\\')[0]

'Megan Fox'

In [21]:
from tmdbv3api import TMDb, Person

# Inicialize a TMDb e configure a chave de API
tmdb = TMDb()
tmdb.api_key = '2034dacd1d3dda0eeea92969115db83d'
tmdb.language = 'pt-BR'

# Crie uma instância da classe Person
person = Person()

# Busque pelo nome do artista

resultado = person.search(artist)

# Exiba o ID do artista
if resultado:
    artista = resultado[0]
    print(f"ID do artista {artista.name}: {artista.id}")
    print([obra['title'] for obra in artista['known_for']])
else:
    print("Artista não encontrado.")

ID do artista Brad Pitt: 287
['Bastardos Inglórios', 'Guerra Mundial Z', 'Clube da Luta']


In [32]:

# Transformando 'results' em um DataFrame principal
df_main = pd.json_normalize(resultado['results'])

# Transformando 'known_for' em DataFrame separado
known_for_data = []
for result in resultado['results']:
    for item in result['known_for']:
        known_for_data.append({
            'name': result['name'],  # Relaciona com o nome do ator
            **item  # Expande as chaves do 'known_for' para o DataFrame
        })

df_known_for = pd.DataFrame(known_for_data)

# Exibindo os DataFrames resultantes
print("DataFrame Principal (Informações do Ator):\n", df_main)
#print("\nDataFrame Conhecido Por (Filmes e Séries):\n", df_known_for)


DataFrame Principal (Informações do Ator):
 Empty DataFrame
Columns: []
Index: [0]


In [31]:
df_known_for

,name,backdrop_path,id,title,original_title,overview,poster_path,media_type,adult,original_language,genre_ids,popularity,release_date,video,vote_average,vote_count
0,Brad Pitt,/1Jpkm9qZcsT0mSyVXgs4VlGjPNI.jpg,16869,Bastardos Inglórios,Inglourious Basterds,"Durante a Segunda Guerra Mundial, na França, u...",/n8fuS2uDDykO4P3RBPKLhWs0Ffj.jpg,movie,False,en,"[18, 53, 10752]",80.095,2009-08-02,False,8.219,22129
1,Brad Pitt,/kg2FOT2Oe5PSCgs3L4vLel6B7ck.jpg,72190,Guerra Mundial Z,World War Z,Um vírus letal se espalha rapidamente e transf...,/2Sdz5gGrwN92pRanlJKeUqkIjRk.jpg,movie,False,en,"[28, 27, 878, 53]",98.794,2013-06-19,False,6.800,15258
2,Brad Pitt,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,550,Clube da Luta,Fight Club,Um homem deprimido que sofre de insônia conhec...,/r3pPehX4ik8NLYPpbDRAh0YRtMb.jpg,movie,False,en,[18],109.714,1999-10-15,False,8.438,29264


In [37]:
pd.DataFrame(resultado['results'][0])

,0
0,adult
1,gender
2,id
3,known_for_department
4,name
5,original_name
6,popularity
7,profile_path
8,known_for


In [41]:
filtered_data = {k: v for k, v in resultado[0].items() if k != "known_for"}

# Convertendo o dicionário em um DataFrame
df = pd.DataFrame([filtered_data])

# Exibindo o DataFrame resultante
df

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path
0,False,2,287,Acting,Brad Pitt,Brad Pitt,85.064,/9OfnD7lxgIj3BNQpJFnwxnwl6w5.jpg
